In [3]:
import tensorflow as tf
import numpy as np
import gc

from skimage.transform import resize
from matplotlib import pyplot as plt
from tqdm import *
%matplotlib inline

IMG_SIZE = 64
DATA_PATH = './data/processed/'

### Data processing

In [2]:
!ls data | grep .npy

test.npy
train-1.npy
train-2.npy
train-3.npy
train-4.npy


In [3]:
dataset_files = ['train-1.npy', 'train-2.npy', 'train-3.npy', 'train-4.npy']
labels_remap = {}

We need to resize all images to same size in order too train network. We also save processed images for future sessions.

In [4]:
def process_data(data, save=False, filename=None):
    # resize all images to [IMG_SIZE x IMG_SIZE]
    # 
    # if `save` is set to True: save processed data to `filename` file
    #
    # returns: 
    #    images -- np.array() of images
    #    labels -- np.array() of image labels
    
    images = []
    labels = []
    

    ind = 0    
    for elem in tqdm(data):
        img, label = elem

        images.append(resize(img, (IMG_SIZE, IMG_SIZE), mode='constant'))
        if label not in labels_remap:
            labels_remap[label] = ind
            ind += 1
        labels.append(labels_remap[label])
    
    images = np.array(images)
    labels = np.array(labels)
    
    if save:
        np.savez(DATA_PATH + filename, images, labels)
            
    return np.array(images), np.array(labels)

In [5]:
x_train, y_train = process_data(np.load('./data/train-1.npy'), save=True, filename='train-1')
# if I want to reuse previously created data
# data = np.load(DATA_PATH + 'train-1.npz')
# x_train = data[data.files[0]]
# y_train = data[data.files[1]]

100%|██████████| 83246/83246 [01:03<00:00, 1301.42it/s]


### Simplest model

For ease of implementation I use Keras

In [1]:
import keras

from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D

# x_train = images
# y_train = labels

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [48]:
x_train = x_train.reshape(x_train.shape[0], IMG_SIZE, IMG_SIZE, 1)
y_train = keras.utils.to_categorical(y_train, num_classes=1000)

In [4]:
model = Sequential()

model.add(Conv2D(16, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(2000, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1000, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [30]:
model.fit(x_train, y_train,
          batch_size=150,
          epochs=10,
          verbose=1)

Epoch 1/10
83246/83246 [==============================] - 342s - loss: 4.3698 - acc: 0.2311   
Epoch 2/10
83246/83246 [==============================] - 330s - loss: 1.3035 - acc: 0.6873   
Epoch 3/10
83246/83246 [==============================] - 337s - loss: 0.6991 - acc: 0.8196   
Epoch 4/10
83246/83246 [==============================] - 328s - loss: 0.4584 - acc: 0.8753   
Epoch 5/10
83246/83246 [==============================] - 321s - loss: 0.3249 - acc: 0.9072   
Epoch 6/10
83246/83246 [==============================] - 322s - loss: 0.2539 - acc: 0.9252   
Epoch 7/10
83246/83246 [==============================] - 357s - loss: 0.2029 - acc: 0.9379   
Epoch 8/10
83246/83246 [==============================] - 339s - loss: 0.1770 - acc: 0.9461   
Epoch 9/10
83246/83246 [==============================] - 320s - loss: 0.1521 - acc: 0.9530   
Epoch 10/10
83246/83246 [==============================] - 321s - loss: 0.1319 - acc: 0.9579   


In [47]:
x_train, y_train = process_data(np.load('./data/train-4.npy'), save=True, filename='train-4')

100%|██████████| 83247/83247 [01:03<00:00, 1303.65it/s]


In [49]:
score = model.evaluate(x_train, y_train, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.327417353323
Test accuracy: 0.919456556994


In [50]:
model.fit(x_train, y_train,
          batch_size=150,
          epochs=5,
          verbose=1)

Epoch 1/5
83247/83247 [==============================] - 323s - loss: 0.5918 - acc: 0.8463   
Epoch 2/5
83247/83247 [==============================] - 320s - loss: 0.3281 - acc: 0.9062   
Epoch 3/5
83247/83247 [==============================] - 318s - loss: 0.2261 - acc: 0.9321   
Epoch 4/5
83247/83247 [==============================] - 318s - loss: 0.1683 - acc: 0.9481   
Epoch 5/5
83247/83247 [==============================] - 318s - loss: 0.1427 - acc: 0.9548   


In [38]:
model.fit(x_train, y_train,
          batch_size=150,
          epochs=1,
          verbose=1)

Epoch 1/1
83247/83247 [==============================] - 350s - loss: 0.1590 - acc: 0.9517   


In [59]:
images = []


ind = 0    
for img in tqdm(np.load('./data/test.npy')):
    images.append(resize(img, (IMG_SIZE, IMG_SIZE), mode='constant'))

images = np.array(images)

100%|██████████| 83247/83247 [01:03<00:00, 1304.40it/s]


In [61]:
x_train = images.reshape(images.shape[0], IMG_SIZE, IMG_SIZE, 1)

In [62]:
pred = model.predict(x_train, verbose=1)

83232/83247 [============================>.] - ETA: 0s

In [66]:
fignafig = np.load('./data/test.npy')[:20]

In [71]:
pred_classes = np.argmax(pred, axis=1)

In [75]:
inv_labels = {v: k for k, v in labels_remap.items()}

In [79]:
ans = []
ind = 1
for cl in pred_classes:
    ans.append([ind, inv_labels[cl]])
    ind += 1

ans = np.array(ans)

In [80]:
ans

array([[    1, 63955],
       [    2, 64432],
       [    3, 64709],
       ..., 
       [83245, 59837],
       [83246, 60602],
       [83247, 59065]])

In [84]:
import pandas as pd
df = pd.DataFrame(ans)

In [ ]:
df.to_csv('pred.csv', index=False, header=['Id', 'Category'])
